In [9]:
# Imports
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score, f1_score, precision_recall_fscore_support
from lightgbm import LGBMClassifier
import shap
import matplotlib.pyplot as plt
import joblib

print('Libraries imported')

Libraries imported


In [15]:
# Configuration
DATA_PATH = r'C:\Users\mk\Downloads\credit_risk_dataset.csv'  # provided dataset path
OUTPUT_DIR = '/mnt/data/credit_risk_shap_outputs'
RANDOM_STATE = 42
TEST_SIZE = 0.2
RANDOM_SEARCH_ITER = 10  # reduce for speed; increase for more thorough tuning
TOP_N_HIGH_RISK = 5  # number of high-risk cases to explain
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [16]:
import os
os.listdir()


['.ipynb_checkpoints', 'Untitled.ipynb']

In [17]:
# Load data
df = pd.read_csv(DATA_PATH)
print('Loaded dataset with shape:', df.shape)
df.head()

Loaded dataset with shape: (32581, 12)


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [18]:
TARGET_COL = "Loan_Status"
print("Using target column:", TARGET_COL)

Using target column: Loan_Status


In [19]:
df.columns.tolist()

['person_age',
 'person_income',
 'person_home_ownership',
 'person_emp_length',
 'loan_intent',
 'loan_grade',
 'loan_amnt',
 'loan_int_rate',
 'loan_status',
 'loan_percent_income',
 'cb_person_default_on_file',
 'cb_person_cred_hist_length']

In [20]:
TARGET_COL = "loan_status"
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]
numeric_feats = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_feats = X.select_dtypes(include=['object', 'category']).columns.tolist()

print("Numeric features:", len(numeric_feats))
print("Categorical features:", len(categorical_feats))

Numeric features: 7
Categorical features: 4


In [22]:
# Basic preprocessing helper to separate features
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

numeric_feats = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_feats = X.select_dtypes(include=['object', 'category']).columns.tolist()
print('Numeric features:', len(numeric_feats))
print('Categorical features:', len(categorical_feats))


Numeric features: 7
Categorical features: 4


In [23]:
# Build preprocessing pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
        ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])
preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, numeric_feats),
        ('cat', categorical_transformer, categorical_feats)
])

print('Preprocessor built')

Preprocessor built


In [24]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_STATE)
print('Train shape:', X_train.shape, 'Test shape:', X_test.shape)


Train shape: (26064, 11) Test shape: (6517, 11)


In [25]:
model = LGBMClassifier(random_state=RANDOM_STATE, n_jobs= -1)
pipe = Pipeline(steps=[('pre', preprocessor), ('model', model)])

param_dist = {
        'model__n_estimators': [100, 300, 600],
        'model__max_depth': [-1, 6, 10],
        'model__learning_rate': [0.01, 0.05, 0.1],
        'model__num_leaves': [31, 63, 127],
        'model__subsample': [0.6, 0.8, 1.0]
}
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
search = RandomizedSearchCV(pipe, param_distributions=param_dist, n_iter=RANDOM_SEARCH_ITER,
                                scoring='roc_auc', cv=cv, random_state=RANDOM_STATE, verbose=2)

print('RandomizedSearchCV configured. Run the next cell to start training (this may take time).')

RandomizedSearchCV configured. Run the next cell to start training (this may take time).


In [26]:
# Fit the randomized search (uncomment to run)
# search.fit(X_train, y_train)
# print('Best params:', search.best_params_)
# best_model = search.best_estimator_
# joblib.dump(best_model, os.path.join(OUTPUT_DIR, 'best_model.joblib'))
# For convenience, if you don't want to run search, train a reasonable default model:
default_pipeline = Pipeline(steps=[('pre', preprocessor), ('model', LGBMClassifier(random_state=RANDOM_STATE, n_estimators=300))])
default_pipeline.fit(X_train, y_train)
best_model = default_pipeline
joblib.dump(best_model, os.path.join(OUTPUT_DIR, 'best_model.joblib'))
print('Trained default model and saved to outputs')


[LightGBM] [Info] Number of positive: 5686, number of negative: 20378
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004549 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 949
[LightGBM] [Info] Number of data points in the train set: 26064, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.218155 -> initscore=-1.276449
[LightGBM] [Info] Start training from score -1.276449
Trained default model and saved to outputs


In [27]:
# Evaluate on test set
y_pred_proba = best_model.predict_proba(X_test)[:,1]
y_pred = (y_pred_proba >= 0.5).astype(int)
print('AUC:', roc_auc_score(y_test, y_pred_proba))
print('\nClassification report (threshold=0.5):')
print(classification_report(y_test, y_pred))
f1_minority = f1_score(y_test, y_pred, pos_label=1)
print('F1 (minority class=1):', f1_minority)


C:\Users\mk\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


AUC: 0.9492773726758398

Classification report (threshold=0.5):
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      5095
           1       0.97      0.73      0.83      1422

    accuracy                           0.94      6517
   macro avg       0.95      0.86      0.90      6517
weighted avg       0.94      0.94      0.93      6517

F1 (minority class=1): 0.8333333333333334


In [28]:
# Prepare a numeric matrix for SHAP (after preprocessing)
pre = best_model.named_steps['pre']
X_train_transformed = pre.transform(X_train)
X_test_transformed = pre.transform(X_test)

# get feature names from transformer
num_names = numeric_feats
cat_names = categorical_feats
feature_names = num_names + cat_names
print('Feature names count:', len(feature_names))


Feature names count: 11


In [29]:
# Create SHAP explainer
lgbm = best_model.named_steps['model']
explainer = shap.TreeExplainer(lgbm)

# Use a small background sample for speed
bg = shap.sample(X_train_transformed, 200, random_state=RANDOM_STATE)
shap_values = explainer.shap_values(X_test_transformed)
print('Computed SHAP values')


Computed SHAP values


C:\Users\mk\AppData\Local\Programs\Python\Python313\Lib\site-packages\shap\explainers\_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(


In [30]:
# Global summary plot
plt.figure(figsize=(8,6))
shap.summary_plot(shap_values, X_test_transformed, feature_names=feature_names, show=False)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'shap_summary_plot.png'), dpi=200)
plt.close()
print('Saved global SHAP summary plot')

Saved global SHAP summary plot


In [31]:
# Feature importance table (mean absolute SHAP)
import numpy as np
mean_abs_shap = np.abs(shap_values).mean(axis=0)
feat_imp = pd.DataFrame({'feature': feature_names, 'mean_abs_shap': mean_abs_shap})
feat_imp = feat_imp.sort_values(by='mean_abs_shap', ascending=False).reset_index(drop=True)
feat_imp.to_csv(os.path.join(OUTPUT_DIR, 'shap_feature_importance.csv'), index=False)
feat_imp.head(20)

,feature,mean_abs_shap
0,loan_percent_income,1.045270
1,person_income,0.965270
2,loan_grade,0.958024
3,person_home_ownership,0.834742
4,loan_intent,0.708206
5,loan_int_rate,0.337943
6,loan_amnt,0.227494
7,person_emp_length,0.146718
8,person_age,0.141391
9,cb_person_cred_hist_length,0.068574


In [32]:
# Dependence plots for top 3 features
top3 = feat_imp['feature'].head(3).tolist()
for f in top3:
        plt.figure(figsize=(6,4))
        # find column index
        idx = feature_names.index(f)
        shap.dependence_plot(idx, shap_values, X_test_transformed, feature_names=feature_names, show=False)
        plt.tight_layout()
        fname = os.path.join(OUTPUT_DIR, f'shap_dependence_{f}.png')
        plt.savefig(fname, dpi=200)
        plt.close()
        print('Saved dependence plot for', f)

Saved dependence plot for loan_percent_income
Saved dependence plot for person_income
Saved dependence plot for loan_grade


<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

In [33]:
# Compute probabilities for the entire test set
probs_test = best_model.predict_proba(X_test)[:,1]
test_df = X_test.copy()
test_df['y_true'] = y_test
test_df['y_proba'] = probs_test
high_risk = test_df[test_df['y_proba'] > 0.8].copy()
print('Found', len(high_risk), 'high-risk cases in the test set')
if len(high_risk) == 0:
        print('No high-risk cases found. Consider lowering the threshold or checking dataset balance.')


C:\Users\mk\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Found 997 high-risk cases in the test set


In [34]:
# Limit to TOP_N_HIGH_RISK cases (largest probabilities)
high_risk = high_risk.sort_values('y_proba', ascending=False).head(TOP_N_HIGH_RISK)

local_explanations = []
for i, (idx, row) in enumerate(high_risk.iterrows()):
        orig_index = idx
        # transformed feature vector for this row
        X_row = pre.transform(X_test.loc[[orig_index]])
        sv = explainer.shap_values(X_row)[0]
        # save waterfall/force plot
        plt.figure(figsize=(6,3))
        try:
            shap.plots._waterfall.waterfall_legacy(explainer.expected_value, sv[0] if sv.ndim>1 else sv, feature_names=feature_names, show=False)
        except Exception:
            # fallback to force_plot saved as image
            shap.force_plot(explainer.expected_value, sv, X_row, feature_names=feature_names, matplotlib=True, show=False)
        plt.tight_layout()
        fname = os.path.join(OUTPUT_DIR, f'shap_local_case_{i+1}.png')
        plt.savefig(fname, dpi=200)
        plt.close()

        # Build structured textual explanation
        # Identify top contributors
        contribs = list(zip(feature_names, sv.flatten() if sv.ndim>1 else sv))
        contribs = sorted(contribs, key=lambda x: abs(x[1]), reverse=True)
        top_contribs = contribs[:5]
        expl_text = {
            'case_rank': i+1,
            'index': int(orig_index) if hasattr(orig_index, 'item') else int(orig_index),
             'predicted_probability': float(row['y_proba']),
            'true_label': int(row['y_true']) if not pd.isna(row['y_true']) else None,
            'top_contributors': [{'feature': f, 'shap_value': float(s)} for f,s in top_contribs]
        }
        local_explanations.append(expl_text)

    
import json
with open(os.path.join(OUTPUT_DIR, 'local_explanations.json'), 'w') as f:
        json.dump(local_explanations, f, indent=2)
print('Saved local explanations and plots for top high-risk cases')

C:\Users\mk\AppData\Local\Programs\Python\Python313\Lib\site-packages\shap\explainers\_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
C:\Users\mk\AppData\Local\Programs\Python\Python313\Lib\site-packages\shap\explainers\_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
C:\Users\mk\AppData\Local\Programs\Python\Python313\Lib\site-packages\shap\explainers\_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
C:\Users\mk\AppData\Local\Programs\Python\Python313\Lib\site-packages\shap\explainers\_tree.py:587: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
C:\Users\mk\AppData\Local\Programs\Python\Python313\Lib\site-packages\shap\explainers\_t

Saved local explanations and plots for top high-risk cases


In [35]:
# Generate a concise technical summary text file describing top 3 drivers and interpretation
summary_lines = []
summary_lines.append('Technical summary of model drivers and SHAP interpretation')
summary_lines.append('Top features by mean absolute SHAP:')
for i, row in feat_imp.head(10).iterrows():
        summary_lines.append(f"{i+1}. {row['feature']} (mean |SHAP| = {row['mean_abs_shap']:.4f})")
summary_lines.append('\nInterpretation guidance:')
summary_lines.append(' - Positive SHAP value increases probability of default; negative SHAP value decreases it.')
summary_lines.append(' - Check feature distributions for potential data leakage or proxies for the target.')
summary_lines.append('\nLocal explanations are saved in local_explanations.json and individual plot PNGs.')
with open(os.path.join(OUTPUT_DIR, 'technical_summary.txt'), 'w') as f:
        f.write('\n'.join(summary_lines))
print('Saved technical summary')


Saved technical summary


In [43]:
# Identify categorical columns
categorical_cols = ["person_home_ownership", "loan_intent", "loan_grade", "cb_person_default_on_file"]

# Convert to category dtype
for col in categorical_cols:
    X[col] = X[col].astype("category")

# Fit the LightGBM model
model.fit(X, y, categorical_feature=categorical_cols)

[LightGBM] [Info] Number of positive: 7108, number of negative: 25473
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003346 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 933
[LightGBM] [Info] Number of data points in the train set: 32581, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.218164 -> initscore=-1.276398
[LightGBM] [Info] Start training from score -1.276398


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [44]:
# 1. Fit the model (after categorical fix)
model.fit(X, y, categorical_feature=["person_home_ownership", "loan_intent", "loan_grade", "cb_person_default_on_file"])

# 2. Get predicted probabilities for the positive class (default = 1)
y_prob = model.predict_proba(X)[:, 1]

# 3. Save model training code
with open("model_training.py", "w") as f:
    f.write("# Model training and SHAP analysis\n")
    f.write("import lightgbm as lgb\nimport shap\nimport pandas as pd\n...\n")  # Add full code here

# 4. Save model architecture and hyperparameters
with open("model_report.txt", "w") as f:
    f.write("Model: LightGBM Classifier\n")
    f.write("Hyperparameters:\n")
    for param, value in model.get_params().items():
        f.write(f"  {param}: {value}\n")

# 5. Save global SHAP interpretation (robust to .values vs array)
try:
    top_features = np.argsort(np.abs(shap_values.values).mean(0))[-3:][::-1]
    shap_array = shap_values.values
except AttributeError:
    top_features = np.argsort(np.abs(shap_values).mean(0))[-3:][::-1]
    shap_array = shap_values

with open("global_shap_summary.txt", "w") as f:
    f.write("Top 3 global SHAP features:\n")
    for i in top_features:
        f.write(f"- {X.columns[i]}\n")

# 6. Save individual SHAP explanations for high-risk cases
high_risk_idx = np.where(y_prob > 0.8)[0][:5]
with open("individual_explanations.txt", "w") as f:
    for i in high_risk_idx:
        f.write(f"\nLoan ID {i} - Predicted Default Probability: {y_prob[i]:.2f}\n")
        top_contrib = np.argsort(np.abs(shap_array[i]))[-3:][::-1]
        for j in top_contrib:
            f.write(f"  {X.columns[j]}: SHAP value = {shap_array[i][j]:.4f}\n")

[LightGBM] [Info] Number of positive: 7108, number of negative: 25473
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008743 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 933
[LightGBM] [Info] Number of data points in the train set: 32581, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.218164 -> initscore=-1.276398
[LightGBM] [Info] Start training from score -1.276398
